In [1]:
import vertexai
from vertexai import model_garden
from google.cloud import aiplatform

vertexai.init(project="qwiklabs-asl-00-85461f18a8ed", location="us-central1")

In [2]:
from PIL import Image
from io import BytesIO
import base64

def base64_to_image(image_str):
    """Convert base64 encoded string to an image."""
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image

In [3]:
flux_endpoint_resource_url="projects/156684330967/locations/us-central1/endpoints/3710863838371905536"

flux_endpoint=aiplatform.Endpoint(flux_endpoint_resource_url)

In [4]:
text = "An oil painting of a robot holding an umbrella over a kid in the rain." 
height = 1024  
width = 1024  
num_inference_steps = 4 

instances = [{"text": text}]
parameters = {
    "height": height,
    "width": width,
    "num_inference_steps": num_inference_steps,
}
response = flux_endpoint.predict(
    instances=instances,
    parameters=parameters,
    use_dedicated_endpoint=True,
)

NotFound: 404 Endpoint projects/156684330967/locations/us-central1/endpoints/3710863838371905536 is not found.

In [ ]:
images=[
    base64_to_image(prediction.get("output"))
        for prediction in response.predictions
]
images[0].save("robot-and-kid.jpg")
display(images[0])

In [ ]:
endpoints = aiplatform.Endpoint.list()

if not endpoints:
    print("No endpoints found in this project/region.")
else:
    for endpoint in endpoints:
        print(f"\nEndpoint: {endpoint.display_name} ({endpoint.resource_name})")

        deployed_models = endpoint.list_models()
        for dm in deployed_models:
            print(f"  Undeploying model ID: {dm.id}")            
            endpoint.undeploy(deployed_model_id=dm.id, sync=True)
            print(f"  Undeployed model ID: {dm.id}")

        print("  Deleting endpoint...")
        endpoint.delete(sync=True)
        print(f"  Deleted endpoint: {endpoint.display_name}")

print("\nDeleting models...")
models = aiplatform.Model.list()

for model in models:
    print(f"  Deleting model: {model.display_name}")
    model.delete(sync=True)
    print("  Deleted.")
